In [1]:
import numpy as np
import astropy.units as u
import astropy.constants as const
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astropy.table import QTable, Table, Column, vstack
import yaml
import re
import os
import time

homedir = os.path.expanduser('~')
savedir = homedir + '/uasal/roman-psf-vetting/data/catalog_tables/'

from astropy.utils.data import clear_download_cache

clear_download_cache()

#Simbad.cache_location

In [2]:
savepath = savedir + 'simbad.fits'

Simbad.add_votable_fields(
    'fluxdata(U)',
    'fluxdata(B)',
    'fluxdata(V)',
    'fluxdata(R)',
    'fluxdata(I)',
    'parallax',
    'sptype',
    'morphtype',
    'measurements',
)

In [3]:
#savedir = save_dir_base + "{}/".format(datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))
if not os.path.isdir(savedir):
    os.makedirs(savedir)
print("Saving to:\n{}".format(savedir))

Saving to:
/home/ntschragal/uasal/roman-psf-vetting/data/catalog_tables/


In [4]:
with open("hd_id_list.yaml", 'r') as f:
    current_list = yaml.safe_load(f)

#print(current_list)

In [5]:
with open("all_v3_stars.yaml", 'r') as f:
    all_v3_stars = yaml.safe_load(f)
    
#print(all_v3_stars)

In [6]:
in_current_list = []

for idx, star in enumerate(all_v3_stars):
    
    in_current_list.append(star in current_list)
    
    
print("V < 3 stars currently in the list: {}".format(np.sum(in_current_list)))
print("Length of the current list:        {}".format(len(current_list)))
print("V < 3 stars cataloged by Simbad:   {}".format(len(all_v3_stars)))

V < 3 stars currently in the list: 61
Length of the current list:        61
V < 3 stars cataloged by Simbad:   181


In [7]:
Simbad.query_object_async("Betelgeuse", cache = False)#, get_query_payload = True)

/home/ntschragal/anaconda3/envs/psfvet/lib/python3.12/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): java.lang.NullPointerException
  warnings.warn("Warning: The script line number %i raised "


TableParseError: Failed to parse SIMBAD result! The raw response can be found in self.last_response, and the error in self.last_table_parse_error. The attempted parsed result is in self.last_parsed_result.
 Exception: No table found in VOTABLE file.

In [8]:
simbad_results = []
min_time = 0.25

for idx, star in enumerate(all_v3_stars):
    print("Processing ({}) {}          ".format(idx + 1, star), end = '\r')
    t0 = time.time()
    simbad_results.append(Simbad.query_object_async(star, cache = False))
    t1 = time.time()
    t_elapsed = t1 - t0
    if t_elapsed < min_time:
        time.sleep(min_time - t_elapsed)

/home/ntschragal/anaconda3/envs/psfvet/lib/python3.12/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): java.lang.NullPointerException
  warnings.warn("Warning: The script line number %i raised "


TableParseError: Failed to parse SIMBAD result! The raw response can be found in self.last_response, and the error in self.last_table_parse_error. The attempted parsed result is in self.last_parsed_result.
 Exception: No table found in VOTABLE file.

In [9]:
simbad_table = vstack(simbad_results)

ValueError: no values provided to stack.

In [ ]:
simbad_table.write(savedir + "simbad_table.fits", format = 'fits')

***

In [20]:
catalogs = {
    'wds': 'B/wds',
    'bourges17': 'II/346',
    'duvert16': 'II/345',
    'stevens17': 'J/AJ/154/259',
    'swihart17': 'J/AJ/153/16',
}

In [21]:
v3_table = Table(
    [all_v3_stars,],
    names = ('id',),
)

#v3_table.show_in_notebook()

In [22]:
cat = catalogs['wds']

wd_companions = []
min_seps = []
min_mags = []

for idx, row in enumerate(v3_table):
    
    name = row['id']
    
    print("Processing ({}) {}          ".format(idx + 1, name), end = '\r')
    
    result = Vizier.query_object(name, catalog = [cat])
    
    if len(result) == 0:
        wd_companions.append(0)
        min_seps.append(np.nan)
        min_mags.append(np.nan)
    else:
        res = result[0]
        wd_companions.append(len(res))
        min_sep = np.amin([np.amin(res['sep1']), np.amin(res['sep2'])])
        min_seps.append(min_sep)
        min_mag = np.amin(res['mag2'])
        min_mags.append(min_mag)

In [26]:
v3_table['wd_comps'] = wd_companions
v3_table['wd_min_comp_sep'] = np.array(min_seps) * u.arcsec
v3_table['wd_min_comp_mag'] = min_mags

In [27]:
#v3_table[v3_table['wd_comps']==0].show_in_notebook()
v3_table.show_in_notebook()

idx,id,wd_comps,wd_min_comp_sep,wd_min_comp_mag
,,,arcsec,
0,HD 123139,2,3.0,14.0
1,HD 207098,2,49.900001525878906,13.140000343322754
2,HD 40312,3,2.0999999046325684,7.199999809265137
3,HD 58350,1,169.0,6.769999980926514
4,HD 215182,9,0.10000000149011612,6.900000095367432
5,HD 44478,5,0.800000011920929,11.289999961853027
6,HD 146051,2,64.5999984741211,11.369999885559082
7,HD 37742,4,0.0,3.700000047683716
8,HD 3712,3,17.600000381469727,8.979999542236328


***

In [26]:
result = Vizier.query_object('Trappist-1', catalog = [cat])

In [27]:
all_v3_stars[2]

'HD 40312'

In [28]:
len(result)

0

In [29]:
?result

Type:        TableList
String form: Empty TableList
Length:      0
File:        ~/anaconda3/envs/psfvet/lib/python3.12/site-packages/astroquery/utils/commons.py
Docstring:  
A class that inherits from `list` but included some pretty printing methods
for an OrderedDict of `astropy.table.Table` objects.

HINT: To access the tables by # instead of by table ID:
>>> t = TableList([('a',1),('b',2)])
>>> t[1]
2
>>> t['b']
2

In [23]:
result[0]

WDS,Disc,Comp,Obs1,Obs2,pa1,pa2,sep1,sep2,mag1,mag2,DM,Notes,n_RAJ2000,RAJ2000,DEJ2000,_RA.icrs,_DE.icrs
,,,yr,yr,deg,deg,arcsec,arcsec,mag,mag,,,,,,,
str10,str7,str5,int16,int16,int16,int16,float32,float32,float32,float32,str9,str4,str1,str11,str11,str11,str11
07241-2918,SMY 2,,1833,2020,285,287,169.0,177.00,2.450,6.77,-29 4328,N,,07 24 05.70,-29 18 11.2,07 24 05.70,-29 18 11.2


***

# Obsolete Code